In [9]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(28,28,3)), # 28x28x1

    layers.Resizing(28,28),
    layers.Rescaling(1./255),
    layers.RandomRotation((-0.2, 0.2)),

    layers.Conv2D(15, kernel_size=(3,3), strides=(1,1), padding='same'), # 10 @ 28x28
    layers.MaxPooling2D((2,2)),

    layers.Flatten(), # 7840
    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    layers.Dropout(0.2),
    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),

    layers.Dense(24, activation=activations.softmax, kernel_initializer=initializers.RandomNormal()) # 24
])

from tensorflow.keras import optimizers, losses, metrics

lr = 0.001

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ metrics.sparse_categorical_accuracy ]
)

from tensorflow.keras import utils

path = './Data'
batch_size = 64

train = utils.image_dataset_from_directory(
    directory=path + '/Train',
    shuffle = True,
    seed = 1,
    image_size = (28,28),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory=path + '/Test',
    shuffle = True,
    seed = 1,
    image_size = (28,28),
    batch_size = batch_size
)

from tensorflow.keras import callbacks

model_path = "./model.keras"

patience = 3
epochs = 100

model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    verbose = True,
    
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks .ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Found 27455 files belonging to 24 classes.
Found 7172 files belonging to 24 classes.
Epoch 1/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - loss: 2.8501 - sparse_categorical_accuracy: 0.1148 - val_loss: 2.4062 - val_sparse_categorical_accuracy: 0.1776
Epoch 2/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - loss: 2.4812 - sparse_categorical_accuracy: 0.2006 - val_loss: 2.1938 - val_sparse_categorical_accuracy: 0.3098
Epoch 3/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - loss: 2.3138 - sparse_categorical_accuracy: 0.2416 - val_loss: 2.0491 - val_sparse_categorical_accuracy: 0.3201
Epoch 4/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - loss: 2.0961 - sparse_categorical_accuracy: 0.3036 - val_loss: 1.7540 - val_sparse_categorical_accuracy: 0.4205
Epoch 5/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - loss: 1.7234 - sparse_categorical_accuracy: 0.4136 - val_loss: 1.4757 - val_sparse_categorical_accuracy: 0.4905
Epoch 6/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - loss: 1.4193 - spar